In [ ]:
%pip install -r requirements.txt

In [1]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv,dotenv_values
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import AzureOpenAI
import os

In [2]:
cosmosdb_endpoint = "<cosmosdb endpoint>"
cosmosdb_key = "<key>"
cosmosdb_connection_str = "string"

cog_search_endpoint = ""
cog_search_key = ""

api_type = "azure"
api_key = ""
api_base = ""
api_version = ""
embeddings_deployment = ""
completions_deployment = ""

In [58]:
client = AzureOpenAI(
  api_key = api_key,
  api_version = api_version,
  azure_endpoint = api_base
)

In [4]:
# Read the PDF file and return the text
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [5]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)


    return chunks

In [42]:
import openai

# Function to generate embeddings using OpenAI's Ada model for text chunks

def generate_embeddings(text):
    response = client.embeddings.create(
    input=text,
    model="serviceswiftada"
    )
    embeddings = response.data[0].embedding
    #response =  openai.Embedding.create(
     #   input=text_chunks, engine="text-embedding-ada-002")
    #embeddings = np.array(response["embeddings"])
    return embeddings


In [81]:
def process(folder_path):
    input_data_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
                        # Get the filename with extension

            print(filename.split('.')[0])
folder_path = "D:\laptop\laptop_ts";
process(folder_path)

ts8
ts9


In [7]:
# Main function to process PDF files, split into chunks, and upload embeddings

def process_pdf_folder(folder_path, chunk_size=1000):
    input_data_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            print(pdf_path)
            text_data = get_pdf_data(pdf_path)
            chunks = get_chunks(text_data, chunk_size)
            file_index = 0

            for chunk in chunks:
                chunk_data = {}
                chunk_data['id'] = f"{filename.split('.')[0]}_{file_index}"
                chunk_data['line'] = chunk
                chunk_data['embedding'] = generate_embeddings(chunk)
                chunk_data['filename'] = pdf_path
                input_data_list.append(chunk_data)
                print(f"Embeddings stored for chunk {file_index} in file {filename}!")

                file_index += 1

    return input_data_list
folder_path = "D:\laptop\laptop_ts";
pdf_data = process_pdf_folder(folder_path)

D:\laptop\laptop_ts\ts8.pdf
Embeddings stored for chunk 0 in file ts8.pdf!
D:\laptop\laptop_ts\ts9.pdf
Embeddings stored for chunk 0 in file ts9.pdf!
Embeddings stored for chunk 1 in file ts9.pdf!
Embeddings stored for chunk 2 in file ts9.pdf!
Embeddings stored for chunk 3 in file ts9.pdf!


In [8]:
# Output embeddings to docVectors.json file
with open("D:/laptop/serviceswift.json", "w") as f:
    json.dump(pdf_data, f)

In [9]:
# Load text-sample.json data file
data_file = open(file="D:/laptop/serviceswift.json", mode="r")
#data_file = open(file="../../DataSet/AzureServices/text-sample_w_embeddings.json", mode="r") # load this file instead if embeddings were previously created and saved.
data = json.load(data_file)
data_file.close()

In [10]:
# Take a peek at one data item
for item_da in data:
    print(item_da)

{'id': 'ts8_0', 'line': 'Troubleshooting connectivity issues using\ncellular settings\nChromebook devices have a SIM card or eSIM which allows you connect to a cellular data network by using a cellular signal.\nIf your cellular data connection is working correctly, you need not change the cellular settings. However, if you are having\ncellular connection issues, changing one or more settings may help.\nFor more information about how to fix cellular connection problems, see Fix connection problems .4\nTroubleshooting connectivity issues using cellular settings 7', 'embedding': [-0.004896602127701044, 0.018050219863653183, 0.025295911356806755, -0.019868044182658195, -0.018818315118551254, 0.0026979318354278803, -0.022274740040302277, -0.027753813192248344, -0.0036292464938014746, -0.015451500192284584, -0.0038180695846676826, 0.006192761939018965, -0.01203348021954298, 0.008743475191295147, 0.011732643470168114, 0.004205317236483097, 0.015233873389661312, -0.022889215499162674, 0.011463

In [11]:
# Generate embeddings for title and content fields
for item in data:
    title = item['id']
    content = item['line']
    file_name = item['filename']
    content_embeddings = item['embedding']
    item['@search.action'] = 'upload'


# Upload data to Azure Cosmos DB


In [12]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey
# Create the client to interact with the Azure Cosmos DB resource
client = CosmosClient(cosmosdb_endpoint, cosmosdb_key)

In [13]:
# Create a database in Azure Cosmos DB.
try:
    database = client.create_database_if_not_exists(id="serviceswift")
    print(f"Database created: {database.id}")

except exceptions.CosmosResourceExistsError:
    print("Database already exists.")

Database created: serviceswift


In [14]:
# Create a container in Azure Cosmos DB.
try:
    partition_key_path = PartitionKey(path="/id")
    container = database.create_container_if_not_exists(
        id="serviceswift_container",
        partition_key=partition_key_path,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: serviceswift_container


In [15]:
# Create data items for every entry in the dataset, insert them into the database and collection specified above.
for data_item in data:
    try:
        container.create_item(body=data_item)
    
    except exceptions.CosmosResourceExistsError:
        print("Data item already exists.")

In [16]:
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

# Create a search index in Cognitive Search

In [17]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
"""from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)"""
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    ComplexField,
    PrioritizedFields,
    SemanticField,
    #SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

In [28]:
# Create index

cog_search_cred = AzureKeyCredential(cog_search_key)
index_name = "serviceswift"

# Create a search index and define the schema (names, types, and parameters)
index_client = SearchIndexClient(
    endpoint=cog_search_endpoint, credential=cog_search_cred)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, 
                key=True, sortable=True, 
                filterable=True, facetable=True),
    SearchableField(name="line", type=SearchFieldDataType.String),
    SearchableField(name="filename", type=SearchFieldDataType.String,
                    filterable=True, facetable=True),
   SearchField(name="embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

# Configure vector search
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        prioritized_content_fields=[SemanticField(field_name="line")],
        prioritized_keywords_fields=[SemanticField(field_name="filename")]
    )
)



# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 serviceswift created


# Create an indexer to pull data from Cosmos DB into Cognitive Search


In [29]:
# Create indexer
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
def _create_datasource():
    # Here we create a datasource. 
    ds_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
    container = SearchIndexerDataContainer(name="serviceswift_container")
    data_source_connection = SearchIndexerDataSourceConnection(
        name="serviceswift", type="cosmosdb", connection_string=cosmosdb_connection_str, container=container
    )
    data_source = ds_client.create_or_update_data_source_connection(data_source_connection)
    return data_source

ds_name = _create_datasource().name
indexer = SearchIndexer(
        name="serviceswift",
        data_source_name=ds_name,
        target_index_name=index_name)

indexer_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
indexer_client.create_or_update_indexer(indexer)  # create the indexer

result = indexer_client.get_indexer("serviceswift")
print(result)

# Run the indexer we just created.
indexer_client.run_indexer(result.name)

{'additional_properties': {'@odata.context': 'https://serviceswiftsearch.search.windows.net/$metadata#indexers/$entity'}, 'name': 'serviceswift', 'description': None, 'data_source_name': 'serviceswift', 'skillset_name': None, 'target_index_name': 'serviceswift', 'schedule': None, 'parameters': None, 'field_mappings': [], 'output_field_mappings': [], 'is_disabled': False, 'e_tag': '"0x8DC2C90606BD312"', 'encryption_key': None, 'cache': None}


In [50]:
from azure.search.documents import SearchClient

search_client = SearchClient(cog_search_endpoint, index_name, cog_search_cred)  
query = "Can't connect to Wi-Fi" 
"""result = generate_embeddings(query)
print(result)"""
results = search_client.search(
    search_text="",  
    vector=Vector(value=generate_embeddings(query), k=3, fields="embedding"),  
    select=["line", "filename"]
)


for result in results:  
    
    print(f"Score: {result['@search.score']}")  
    print(f"Captions: {result['@search.captions']}")  
    print(f"Highlights: {result['@search.highlights']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.8293828
Captions: None
Highlights: None
Content: Troubleshooting connectivity issues using
cellular settings
Chromebook devices have a SIM card or eSIM which allows you connect to a cellular data network by using a cellular signal.
If your cellular data connection is working correctly, you need not change the cellular settings. However, if you are having
cellular connection issues, changing one or more settings may help.
For more information about how to fix cellular connection problems, see Fix connection problems .4
Troubleshooting connectivity issues using cellular settings 7

Filename: D:\laptop\laptop_ts\ts8.pdf

###############################
Score: 0.7810859
Captions: None
Highlights: None
Content:  When the
system will no longer power on when the power button is pressed, the battery is fully discharged.
●Do not crush, drop, mutilate, or penetrate the battery with foreign objects.
●Do not expose the battery to high temperatures, or disassemble battery packs and cells.


In [51]:
def vector_search(q):
    search_client = SearchClient(cog_search_endpoint, index_name, cog_search_cred)  
   # query = "Can't connect to Wi-Fi" 
    """result = generate_embeddings(query)
    print(result)"""
    results = search_client.search(
        search_text="",  
        vector=Vector(value=generate_embeddings(q), k=3, fields="embedding"),  
        select=["line", "filename"]
    )
    return results


q = "Can't connect to Wi-Fi" 
resp =  vector_search(q)
for result in resp:  
    
    print(f"Score: {result['@search.score']}")  
    print(f"Captions: {result['@search.captions']}")  
    print(f"Highlights: {result['@search.highlights']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.8293828
Captions: None
Highlights: None
Content: Troubleshooting connectivity issues using
cellular settings
Chromebook devices have a SIM card or eSIM which allows you connect to a cellular data network by using a cellular signal.
If your cellular data connection is working correctly, you need not change the cellular settings. However, if you are having
cellular connection issues, changing one or more settings may help.
For more information about how to fix cellular connection problems, see Fix connection problems .4
Troubleshooting connectivity issues using cellular settings 7

Filename: D:\laptop\laptop_ts\ts8.pdf

###############################
Score: 0.7810859
Captions: None
Highlights: None
Content:  When the
system will no longer power on when the power button is pressed, the battery is fully discharged.
●Do not crush, drop, mutilate, or penetrate the battery with foreign objects.
●Do not expose the battery to high temperatures, or disassemble battery packs and cells.


In [71]:
#This function helps 

def generate_completion(prompt):
    system_prompt = '''
   You are a personal Bot assistant for answering any questions about documents of uploaded manuals.
    You are given a question and a set of documents.
    If the user's question requires you to provide specific information from the documents, 
    give your answer based only on the examples provided below. DON'T generate an answer 
    that is NOT written in the provided examples.
    If you don't find the answer to the user's question with the examples provided to you below, 
    answer that you dont have knowledge  about the question  and propose him to rephrase his query with more details.
    Use bullet points if you have to make a list, only if necessary.  "
    '''

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]

    for item in results:
        messages.append({"role": "system", "content": prompt['content']})

    response = client.chat.completions.create(
    model=completions_deployment,
    messages=messages,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    
    return response

In [68]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about assistance. Type 'end' to end the session.\n")
user_input = input("Prompt: ")
while user_input.lower() != "end":
    results_for_prompt = vector_search(user_input)
    completions_results = generate_completion(results_for_prompt)
    print("\n")
    print(completions_results.choices[0].message.content)
    user_input = input("Prompt: ")


*** Please ask your model questions about assistance. Type 'end' to end the session.

Prompt: Troubleshooting connectivity issues using cellular settings


To troubleshoot connectivity issues using cellular settings, you can try the following steps:

1. Check signal strength: Ensure that you have a strong cellular signal by checking the signal bars on your device. If the signal is weak, try moving to a different location or contacting your cellular service provider for assistance.

2. Enable and disable airplane mode: Turn on airplane mode for a few seconds and then turn it off. This can help refresh the cellular connection.

3. Restart your device: Sometimes, a simple restart can resolve connectivity issues. Turn off your device, wait for a few seconds, and then turn it back on.

4. Check cellular data settings: Make sure that cellular data is enabled on your device. Go to the settings menu and look for the cellular or mobile data option. Ensure that it is turned on.

5. Reset network

KeyboardInterrupt: Interrupted by user

In [ ]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about assistance. Type 'end' to end the session.\n")
user_input = input("Prompt: ")
while user_input.lower() != "end":
    results_for_prompt = vector_search(user_input)
    completions_results = generate_completion(results_for_prompt)
    print("\n")
    print(completions_results.choices[0].message.content)
    user_input = input("Prompt: ")


*** Please ask your model questions about assistance. Type 'end' to end the session.

Prompt: My HP ENVY x360 2-in-1 touchscreen is so heated. It is a tricky situation.


It seems like you are experiencing overheating issues with your HP ENVY x360 2-in-1 touchscreen. Here are a few possible solutions based on the information provided:

- Ensure that the laptop is placed on a flat and hard surface to allow proper airflow.
- Check if the laptop's vents are not blocked by dust or debris. If they are, clean them using compressed air or a soft brush.
- Update the laptop's BIOS and drivers to the latest versions, as these updates often include improvements for heat management.
- Reduce the number of running programs or processes that may be causing excessive heat generation.
- Consider using a laptop cooling pad or external fan to help dissipate heat more effectively.
- If the issue persists, it may be necessary to contact HP support for further assistance.

If you have any additional detail